### HomeWork3

In [26]:
import numpy as np
import cv2

In [58]:
cap = cv2.VideoCapture(0)
# cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
# cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)

while True:
    ret, frame = cap.read()
#     roi = frame[0:500, 0:500]
    # 가져온 이미지의 일부분 만 가져와서 회색으로 표현
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 가우시안 블러를 통해 노이즈 제거
    gray_blur = cv2.GaussianBlur(gray, (15, 15), 0)

    
    # 그 후 임계처리(임계값을 기준으로 흑백 나누는것)
    thresh = cv2.adaptiveThreshold(
        gray_blur, # 블러처리한 이미지
        255, # 임계값
        cv2.ADAPTIVE_THRESH_GAUSSIAN_C, # 처리할 방법(여기선 가우시안)
        cv2.THRESH_BINARY_INV, #오로지 흑백으로 하는건데 역전된거
        11, # 블럭 사이즈
        1
    )
    
    # 커널의 값에 따라 검은색이 흰색으로 바뀌는 정도가 달라짐
    kernel = np.ones((3, 3), np.uint8)
    # Dilation(오브젝트 외곽선 흰색) 연산 후 Erosion(흰색 오브젝트의 외곽선 검은색)연산 진행  
    # --> 흰색 오브젝트에 있는 검은색을 흰색으로 변경
    closing = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel, iterations=4)
    
    cont_img = closing.copy()
    contours, hierarchy = cv2.findContours( # 윤곽선 검색후 윤곽선과 계층정보(속성 정보) 반환
        cont_img, # 이미지
        cv2.RETR_EXTERNAL, # 검색방법 (외곽윤각선만 검출 , 계층정보 구성하지 않음)
        cv2.CHAIN_APPROX_SIMPLE) # 근사화 방법?
    
    for cnt in contours:
        area = cv2.contourArea(cnt) # 검출한 외곽선의 너비
        if area < 2000 or area > 3000: # 면적이 일정수준을 넘어가거나 못미치면 패스
            continue
        if len(cnt) < 3: 
            continue
        ellipse = cv2.fitEllipse(cnt) # 검출된 외곽선을 감싸는 타원 반환
        cv2.ellipse(frame, ellipse, (0,255,0), 2) # frame에 그려주기
    
    
    
    cv2.imshow('frame',frame)
    
    cv2.imshow('test1', gray_blur)
    
    cv2.imshow('thresh',thresh)
    cv2.imshow('closing', closing)
    k = cv2.waitKey(5) & 0xFF
    if k == 27:
        break

cap.release()
cv2.destroyAllWindows()